In [1]:
import sys
import os

root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from service.config import INTERACTIONS_DATA, USER_DATA, ITEM_DATA

from lightfm import LightFM

from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel
from rectools.dataset import Dataset
from rectools import Columns
from rectools.metrics import calc_metrics, Precision, Recall, Serendipity, MeanInvUserFreq, MAP, NDCG
from catboost import CatBoostRanker, CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from typing import Dict
import pickle


/Users/nikitareznikov/jupyter_notebook_venv/jupyter_venv/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
interactions = pd.read_csv(INTERACTIONS_DATA)
interactions.rename(
        columns={
                'last_watch_dt': 'datetime',
                'total_dur': 'weight'
        }, 
        inplace=True)
interactions['datetime'] = pd.to_datetime(interactions['datetime'])


users = pd.read_csv(USER_DATA)
items = pd.read_csv(ITEM_DATA)

In [4]:
def apply_tfidf_transform(input_dataframe, feature_column, features_limit=500):
    tfidf_creator = TfidfVectorizer(max_features=features_limit)
    tfidf_data = tfidf_creator.fit_transform(input_dataframe[feature_column])
    column_labels = [f'{feature_column}_feature_{idx}' for idx in range(tfidf_data.shape[1])]
    return pd.DataFrame(tfidf_data.toarray(), columns=column_labels)

In [5]:
for category in ['genres', 'keywords']:
    items[category].fillna('', inplace=True)
    transformed_data = apply_tfidf_transform(items, category)
    items = pd.concat([items, transformed_data], axis=1)
    items.drop(columns=category, inplace=True)

In [16]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами 
# из датафрейма с взаимодействиями
users_ohe_df = users.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом, 
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()


,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,sex_Ж,sex_М,kids_flg_0,kids_flg_1
0,973171,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1
1,962099,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0
2,1047345,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0
3,721985,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0
4,704055,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0


In [5]:
max_date = interactions['datetime'].max()
min_date = interactions['datetime'].min()

print(f'min дата в interactions: {min_date}')
print(f'max дата в interactions: {max_date}')
print(f'Продолжительность: {max_date - min_date}')

min дата в interactions: 2021-03-13 00:00:00
max дата в interactions: 2021-08-22 00:00:00
Продолжительность: 162 days 00:00:00


In [6]:
ranker_days_count = 30

ranker_data = interactions[
    (interactions['datetime'] >= max_date - pd.Timedelta(days=ranker_days_count))
]

train_size = 0.7
val_size = 0.15
test_size = 0.15


train_val_users, test_users = train_test_split(
    ranker_data['user_id'].unique(), random_state=42, test_size=test_size
)

train_users, val_users = train_test_split(
    train_val_users, random_state=42, test_size=val_size / (train_size + val_size)  # 15% от общего размера
)

base_models_data = interactions[
    (interactions['datetime'] < max_date - pd.Timedelta(days=ranker_days_count))
]

In [15]:
RANDOM_STATE = 42
NUM_THREADS = 16
K_RECOS = 100


if os.path.exists('ligtfm_predicts.csv'):
    candidates = pd.read_csv('ligtfm_predicts.csv').drop(columns=['Unnamed: 0'])
else:
    dataset = Dataset.construct(base_models_data)

    model = LightFMWrapperModel(
                LightFM(
                    no_components=10, 
                    loss='warp', 
                    random_state=RANDOM_STATE,
                    learning_rate=0.05
                ),
                epochs=10,
                num_threads=NUM_THREADS,
            )

    model.fit(dataset)
    
    candidates = model.recommend(
        users=base_models_data['user_id'].unique(),
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True
    )
    candidates.to_csv('ligtfm_predicts.csv')


In [11]:
# Считаем метрики
def calc_metrics_(candidates_df, rank_col: str) -> Dict[str, float]:
    metrics = {
        'ndcg@10': NDCG(k = 10),
        'map@10': MAP(k = 10),
        'Precision@10': Precision(k = 10),
        'recall@10': Recall(k = 10),
        'novelty@10': MeanInvUserFreq(k = 10),
    }
    return calc_metrics(
        metrics=metrics,
        reco=(
            candidates_df
            .rename(columns={rank_col: Columns.Rank})
            [[Columns.User, Columns.Item, Columns.Rank]]
            [candidates_df[Columns.User].isin(test_users)]
        ),
        interactions=(
            ranker_data
            [[Columns.User, Columns.Item, Columns.Datetime, Columns.Weight]]
            [ranker_data[Columns.User].isin(test_users)]
        ), 
        prev_interactions=(
            base_models_data
            [[Columns.User, Columns.Item, Columns.Datetime, Columns.Weight]]
            [base_models_data[Columns.User].isin(test_users)]
        ),
        catalog=items['item_id'].unique()
    )

models_metrics: Dict[str, Dict[str, float]] = dict()
models_metrics['lfm'] = calc_metrics_(candidates, 'rank')
models_metrics['lfm']

{'Precision@10': 0.02601568615412951,
 'recall@10': 0.08352774152679072,
 'ndcg@10': 0.03275154186487149,
 'map@10': 0.04228483913150633,
 'novelty@10': 3.735845879554382}

In [8]:
# Получаем длину истории юзера 
base_models_data['user_hist'] = (
    base_models_data.groupby('user_id')
    ['item_id'].transform('count')
)
# Получаем популярность контента
base_models_data['item_pop'] = (
    base_models_data.groupby('item_id')
    ['user_id'].transform('count')
)
# Получаем среднюю популярность контента, просматриваемого этим юзером
base_models_data['user_avg_pop'] = (
    base_models_data.groupby('user_id')
    ['item_pop'].transform('mean')
)
# Получаем среднюю длину истории пользователя, которые смотрит этот контент
base_models_data['item_avg_hist'] = (
    base_models_data.groupby('item_id')
    ['user_hist'].transform('mean')
)
# Получаем популярность последнего просмотренного контента
base_models_data.sort_values(
    by=[Columns.User, Columns.Datetime], 
    ascending=[True, False], 
    ignore_index=True,
    inplace=True,
)
base_models_data['user_last_pop'] = (
    base_models_data.groupby('user_id')
    ['item_pop'].transform('first')
)
base_models_data.head(3)

/var/folders/xw/93749g452yg2xp53m3yrxghc0000gn/T/ipykernel_8910/3643323132.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_models_data['user_hist'] = (
/var/folders/xw/93749g452yg2xp53m3yrxghc0000gn/T/ipykernel_8910/3643323132.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_models_data['item_pop'] = (
/var/folders/xw/93749g452yg2xp53m3yrxghc0000gn/T/ipykernel_8910/3643323132.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,user_id,item_id,datetime,weight,watched_pct,user_hist,item_pop,user_avg_pop,item_avg_hist,user_last_pop
0,0,6006,2021-07-20,1,0.0,6,5208,41885.0,16.891897,5208
1,0,7102,2021-07-19,169,3.0,6,11626,41885.0,20.349475,5208
2,0,14359,2021-07-19,130,2.0,6,6053,41885.0,22.546836,5208


In [21]:
users = pd.merge(
    left=users, 
    right=(
        base_models_data
        [['user_id', 'user_hist', 'user_avg_pop', 'user_last_pop']]
        .drop_duplicates()
    ),
    how='left',
    on='user_id',
)
users.head(3)

,user_id,age,income,sex,kids_flg,user_hist,user_avg_pop,user_last_pop
0,973171,age_25_34,income_60_90,М,1,5.0,19550.800000,93403.0
1,962099,age_18_24,income_20_40,М,0,13.0,1329.307692,260.0
2,1047345,age_45_54,income_40_60,Ж,0,NaN,NaN,NaN


In [29]:
default_values_items = {
    'release_year': items['release_year'].median(),
    'age_rating': items['age_rating'].median(),
    'item_pop': base_models_data['item_pop'].median(),
    'item_avg_hist': base_models_data['item_avg_hist'].median()
}

default_values_users = {
    'user_hist': 0,
    'user_avg_pop': base_models_data['user_avg_pop'].median(),
    'user_last_pop': base_models_data['user_last_pop'].median(),
    'income': 'Unknown',
    'age': 'Unknown',
    'sex': 'Unknown',
    'kids_flg': 0
}

interactions_default_values = {
   Columns.Datetime: interactions[Columns.Datetime].median(),
    Columns.Weight: 0.,
    'total_dur': 0,
}

In [30]:

def users_filter(
    user_list: np.ndarray,
    candidates_df: pd.DataFrame, 
    df: pd.DataFrame,
) -> pd.DataFrame:
    # Джойним интеракции на наших кандидатов для юзеров из трейна, вал и теста
    df = pd.merge(
        df[df['user_id'].isin(user_list)], 
        candidates_df[candidates_df['user_id'].isin(user_list)], 
        how='outer',  # right ? 
        on=['user_id', 'item_id']
    )
    # Проставляем дефолтные значения интеракций
    min_score: float =  df['score'].min() - 0.01
    max_rank: int = df['rank'].max() + 1  # 101
    
    default_values = {
        'score': min_score, 'rank': max_rank,
        **interactions_default_values,
    }
    df.fillna(default_values, inplace=True)
    df.sort_values(
        by=['user_id', 'item_id'],
        inplace=True,
    )
    df.dropna(subset=['watched_pct'], inplace=True)
    return df

ranker_train = users_filter(train_users, candidates, ranker_data)
ranker_val = users_filter(val_users, candidates, ranker_data)
ranker_test = users_filter(test_users, candidates, ranker_data)

ranker_train.head(3)

/var/folders/xw/93749g452yg2xp53m3yrxghc0000gn/T/ipykernel_8910/1310415032.py:21: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.fillna(default_values, inplace=True)
/var/folders/xw/93749g452yg2xp53m3yrxghc0000gn/T/ipykernel_8910/1310415032.py:21: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.fillna(default_values, inplace=True)
/var/folders/xw/93749g452yg2xp53m3yrxghc0000gn/T/ipykernel_8910/1310415032.py:21: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always sett

,user_id,item_id,datetime,weight,watched_pct,score,rank
1180642,3,47,2021-08-16,2179.0,27.0,-6.482238,101.0
998971,3,142,2021-08-13,5892.0,100.0,2.700313,10.0
1067328,3,965,2021-08-16,5813.0,96.0,-6.482238,101.0


In [31]:
# Добавляем фичи
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.merge(
        df, 
        users, 
        how='left', 
        on=['user_id']
    )
    df.fillna(default_values_users, inplace=True)
    return df

ranker_train = add_features(ranker_train)
ranker_val = add_features(ranker_val)
ranker_test = add_features(ranker_test)

ranker_train.head(3)

,user_id,item_id,datetime,weight,watched_pct,score,rank,age,income,sex,kids_flg,user_hist,user_avg_pop,user_last_pop
0,3,47,2021-08-16,2179.0,27.0,-6.482238,101.0,Unknown,Unknown,Unknown,0.0,0.0,11957.864865,2858.0
1,3,142,2021-08-13,5892.0,100.0,2.700313,10.0,Unknown,Unknown,Unknown,0.0,0.0,11957.864865,2858.0
2,3,965,2021-08-16,5813.0,96.0,-6.482238,101.0,Unknown,Unknown,Unknown,0.0,0.0,11957.864865,2858.0


In [35]:
features = [
    'weight',
    'score',
    'age',
    'income',
    'sex',
    'kids_flg',
    'user_hist',
    'user_avg_pop',
    'user_last_pop'
]

cat_features = [
    'age',
    'income',
    'sex'
]

target = 'watched_pct'

In [40]:
rank_model = CatBoostRegressor()

rank_model.fit(
    X=ranker_train[features],
    y=ranker_train[target],
    cat_features=cat_features
)

Learning rate set to 0.125808
0:	learn: 35.5808675	total: 353ms	remaining: 5m 52s
1:	learn: 32.6135313	total: 586ms	remaining: 4m 52s
2:	learn: 30.1277930	total: 788ms	remaining: 4m 21s
3:	learn: 28.0736982	total: 1.04s	remaining: 4m 19s
4:	learn: 26.3815375	total: 1.24s	remaining: 4m 7s
5:	learn: 25.0150987	total: 1.43s	remaining: 3m 56s
6:	learn: 23.9100612	total: 1.6s	remaining: 3m 46s
7:	learn: 23.0325702	total: 1.88s	remaining: 3m 53s
8:	learn: 22.3183871	total: 2.09s	remaining: 3m 49s
9:	learn: 21.7507441	total: 2.27s	remaining: 3m 44s
10:	learn: 21.2988173	total: 2.42s	remaining: 3m 37s
11:	learn: 20.9464910	total: 2.63s	remaining: 3m 36s
12:	learn: 20.6722144	total: 2.83s	remaining: 3m 35s
13:	learn: 20.4538323	total: 3.01s	remaining: 3m 32s
14:	learn: 20.2817074	total: 3.15s	remaining: 3m 27s
15:	learn: 20.1483334	total: 3.33s	remaining: 3m 25s
16:	learn: 20.0430559	total: 3.52s	remaining: 3m 23s
17:	learn: 19.9563576	total: 3.7s	remaining: 3m 21s
18:	learn: 19.8899946	total: 

In [49]:
def rerank(
        df: pd.DataFrame,
        first_ranks_col: str,
        second_ranks: np.array,
    ):
    df = df.copy()
    df['second_model_scores'] = second_ranks
    mask = (df[first_ranks_col] < 101).to_numpy()
    df['hybryd_score'] = df['second_model_scores'] * mask
    min_score: float = min(second_ranks) - 0.01
    df[f'hybryd_score'].replace(
        0,
        min_score,
        inplace=True,
    )
    df.sort_values(
        by=['user_id', f'hybryd_score'],
        ascending=[True, False],
        inplace=True,
    )
    df['hybryd_rank'] = df.groupby('user_id').cumcount() + 1
    df['hybryd_rank'] = df['hybryd_rank'] * mask
    max_rank = 101
    df['hybryd_rank'].replace(
        0,
        max_rank,
        inplace=True,
    )
    return df


In [50]:
ranker_val.head()

,user_id,item_id,datetime,weight,watched_pct,score,rank,age,income,sex,kids_flg,user_hist,user_avg_pop,user_last_pop
0,21,3784,2021-07-26,675.0,13.0,-2.527734,101.0,age_45_54,income_20_40,Ж,0.0,26.0,14222.346154,1988.0
1,21,12360,2021-08-15,127.0,2.0,-2.527734,101.0,age_45_54,income_20_40,Ж,0.0,26.0,14222.346154,1988.0
2,21,13787,2021-08-18,2355.0,42.0,-2.527734,101.0,age_45_54,income_20_40,Ж,0.0,26.0,14222.346154,1988.0
3,21,14488,2021-08-18,6396.0,100.0,-2.527734,101.0,age_45_54,income_20_40,Ж,0.0,26.0,14222.346154,1988.0
4,26,4135,2021-08-20,17721.0,84.0,-2.527734,101.0,age_25_34,income_60_90,М,1.0,0.0,11957.864865,2858.0


In [51]:
predicts = rank_model.predict(
    ranker_val[features]
)

rerank_val = rerank(ranker_val, 'rank', predicts)

# Делаем предсказания для всех пользователей

In [4]:
RANDOM_STATE = 42
NUM_THREADS = 16
K_RECOS = 100

all_dataset = Dataset.construct(interactions)

if os.path.exists('lightfm_recommendations.csv'):
    recs = pd.read_csv('lightfm_recommendations.csv')
else:
    model = LightFMWrapperModel(
                LightFM(
                    no_components=10, 
                    loss='warp', 
                    random_state=RANDOM_STATE,
                    learning_rate=0.05
                ),
                epochs=10,
                num_threads=NUM_THREADS,
            )

    model.fit(all_dataset)
    # Делаем рекомендации
    recs = model.recommend(
        users=interactions['user_id'].unique(),
        dataset=all_dataset,
        k=K_RECOS,
        filter_viewed=True
    )

    recs.to_csv('lightfm_recommendations.csv', index=False)

In [5]:
# Удаляем лишние столбцы
items.drop(
    columns=[
        'title',
        'title_orig',
        'for_kids',
        'actors',
        'description',
        'directors',
        'studios',
        'keywords',
        'countries',
        'genres'
    ],
    inplace=True
)

In [5]:
interactions_ranked = interactions.sample(n=3000000, random_state=42)

In [6]:
# Подготавливаем данные для реранжирования
reranking_train_df = interactions_ranked.merge(users, on='user_id')
reranking_train_df = reranking_train_df.merge(items, on='item_id')
reranking_train_df.head()

,user_id,item_id,datetime,weight,watched_pct,age,income,sex,kids_flg,content_type,release_year,genres,countries,age_rating,studios,directors,keywords
0,772446,14431,2021-03-17,7113,100.0,age_18_24,income_20_40,Ж,0,film,2021.0,"ужасы, триллеры, мелодрамы",Россия,16.0,NaN,Святослав Подгаевский,", 2021, россия, приворот, чёрное, венчание"
1,60502,14431,2021-04-17,2616,47.0,age_45_54,income_20_40,М,0,film,2021.0,"ужасы, триллеры, мелодрамы",Россия,16.0,NaN,Святослав Подгаевский,", 2021, россия, приворот, чёрное, венчание"
2,297409,14431,2021-06-24,5386,97.0,age_55_64,income_20_40,М,0,film,2021.0,"ужасы, триллеры, мелодрамы",Россия,16.0,NaN,Святослав Подгаевский,", 2021, россия, приворот, чёрное, венчание"
3,145813,14431,2021-06-15,6022,100.0,age_45_54,income_40_60,Ж,1,film,2021.0,"ужасы, триллеры, мелодрамы",Россия,16.0,NaN,Святослав Подгаевский,", 2021, россия, приворот, чёрное, венчание"
4,574046,14431,2021-06-17,9730,100.0,age_45_54,income_60_90,Ж,0,film,2021.0,"ужасы, триллеры, мелодрамы",Россия,16.0,NaN,Святослав Подгаевский,", 2021, россия, приворот, чёрное, венчание"


In [6]:
droped_columns = [
    'user_id',
    'item_id',
    'datetime',
    'weight',
    'watched_pct',
]

category_features = [
    'age',
    'income',
    'sex',
    'content_type',
]

target = ['watched_pct']

if os.path.exists('reranking_model.pkl'):
    with open('reranking_model.pkl', 'rb') as f:
        ranked_model = pickle.load(f)
else:
    ranked_model = CatBoostRegressor()

    reranking_train_df.fillna({elem: 'Unknown' for elem in category_features}, inplace=True)
    ranked_model.fit(
        X=reranking_train_df.drop(columns=droped_columns),
        y=reranking_train_df[target].fillna(0),
        cat_features=category_features,
    )

    with open('reranking_model.pkl', 'wb') as f:
        pickle.dump(ranked_model, f)

In [7]:
rerankind_recs = recs.merge(users, on='user_id').merge(items, on='item_id')
rerankind_recs_for_predict = rerankind_recs.drop(columns=['user_id', 'item_id', 'score', 'rank'])
rerankind_recs_for_predict.fillna({elem: 'Unknown' for elem in category_features}, inplace=True)
rerankind_recs_for_predict.head()

,age,income,sex,kids_flg,content_type,release_year,age_rating
0,age_35_44,income_40_60,М,0,film,2019.0,16.0
1,age_25_34,income_20_40,Ж,0,film,2019.0,16.0
2,age_18_24,income_40_60,Ж,0,film,2019.0,16.0
3,age_45_54,income_20_40,М,0,film,2019.0,16.0
4,age_18_24,income_20_40,Ж,0,film,2019.0,16.0


In [8]:
predicts = ranked_model.predict(rerankind_recs_for_predict)

In [9]:
rerankind_recs['new_score'] = predicts

In [11]:
df_for_save = rerankind_recs[['user_id', 'item_id', 'new_score']]
df_for_save['new_rank'] = df_for_save.sort_values(by='new_score', ascending=False).groupby('user_id').cumcount() + 1
df_for_save = df_for_save[df_for_save['new_rank'] <= 10]
df_for_save.head()

/var/folders/xw/93749g452yg2xp53m3yrxghc0000gn/T/ipykernel_13500/2309429520.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_save['new_rank'] = df_for_save.sort_values(by='new_score', ascending=False).groupby('user_id').cumcount() + 1


,user_id,item_id,new_score,new_rank
311794,594274,5693,64.107929,10
335048,845671,5693,64.107929,10
342893,887912,1785,52.435934,8
345240,471680,1785,51.738632,9
347581,781277,1785,50.157831,10


In [12]:
df_for_save.to_csv('result_predicts.csv', index=False)

In [13]:
users_recs = df_for_save.groupby('user_id').agg({'item_id': list})
users_recs.to_json('lightfm_recommendations.json')